In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.utils import resample
from xgboost import XGBRegressor
from difflib import SequenceMatcher

In [2]:
def clean(DataFrame):
    
    drop_list=[]

    for column in DataFrame.columns:
        if DataFrame[column].isna().sum()/len(DataFrame) > 0.05:
            drop_list.append(column)   
    
    for column in DataFrame.columns:
        if (len(DataFrame[column].value_counts())==1)&(column not in drop_list):
            drop_list.append(column)

    drop_list_id=['ASIN', 'Valoración media de los clientes', 'Clasificación en los más vendidos de Amazon','Referencia del fabricante','Nmero de modelo del producto', 'Clasificacin en los ms vendidos de Amazon', 'Producto en Amazon.es desde', 'Dimensiones del paquete']
    drop_list= drop_list+drop_list_id
    drop_list.append('Dimensiones del producto')
    drop_list_repeated=['Peso del producto','Dimensiones del producto: largo x ancho x alto']
    drop_list= drop_list+drop_list_repeated
    
    size=[]
    weight=[]
    
    for value in DataFrame['Dimensiones del producto']:
        if ';' in str(value):
            size.append(str(value).split(';')[0])
            weight.append(str(value).split(';')[1])
        else:
            size.append(str(value))
            weight.append(np.nan)

    
    largo=[]
    largo_clean=[]
    alto_clean=[]
    weight_clean=[]
    ancho=[]
    alto=[]
    size_sep=[]
    string='\u200e'
    
    
    for i in size:
        if i=='nan':
            largo.append(i)
            ancho.append(i)
            alto.append(i)
        else:
            largo.append(i.split('x')[0])
            ancho.append(i.split('x')[1])
            alto.append(i.split('x')[2])
    
    
    for i in weight:
        if pd.isna(i):
            weight_clean.append(i)
        else:
            weight_clean.append(str(i).split(' ')[1])
            y=i.split(' ')[2]
            
    
    
    for i in alto:
        if i== 'nan':
            alto_clean.append(i)
        else:
            alto_clean.append(str(i).split(' ')[1])
            x=i.split(' ')[2]
    
    for i in largo:
        if string in i:
            value = i[len(string):]
            largo_clean.append(value)
        else:
            largo_clean.append(i)
    
   
    for i in range(0, len(largo_clean)):
        largo_clean[i] = float(largo_clean[i])
    for i in range(0, len(ancho)):
        ancho[i] = float(ancho[i])
    for i in range(0, len(alto_clean)):
        alto_clean[i] = float(alto_clean[i])
    for i in range(0, len(weight_clean)):
        weight_clean[i] = float(weight_clean[i])    

    DataFrame['length '+str(x)]=largo_clean
    DataFrame['width '+str(x)]=ancho
    DataFrame['height '+str(x)]=alto_clean
    DataFrame['weight '+str(y)]=weight_clean
    
    DataFrame['length '+str(x)]=DataFrame['length '+str(x)].fillna(np.mean(largo_clean))
    DataFrame['width '+str(x)]=DataFrame['width '+str(x)].fillna(np.mean(ancho))
    DataFrame['height '+str(x)]=DataFrame['height '+str(x)].fillna(np.mean(alto_clean))
    DataFrame['weight '+str(y)]=DataFrame['weight '+str(y)].fillna(np.mean(weight_clean))
    

    DataFrame=DataFrame.reset_index(drop=True)
    
    def similar(DataFrame):
        new_drop=[]
        columns=[]
        ratio=[]
        for column in DataFrame.columns:
            if column != 'Tamaño':
                if SequenceMatcher(None, DataFrame['Tamaño'][0], str(DataFrame[column][0])).ratio() > 0.5:
                    new_drop.append(column)
            
        return new_drop
    
    dic_NAN=dict(zip(DataFrame.columns,DataFrame.isna().sum()/len(DataFrame)))
    list_NAN=sorted(dic_NAN, key=dic_NAN.get, reverse=True)
    DataFrame=DataFrame[DataFrame[list_NAN[0]].isna()==False]
    DataFrame=DataFrame.reset_index(drop=True)    
    
    
    drop_list= drop_list+ similar(DataFrame)
    
    for column in drop_list:
        if column in drop_list:
            DataFrame=DataFrame.drop(column,axis=1)

    
    return DataFrame



In [18]:
def output(df):
    
    X=df.drop(['Rating'],axis=1)
    y=df['Rating']
    

    X_cat=X.select_dtypes(include='object')
    X_num=X.select_dtypes(include='number')
    X_cat_encoded= pd.get_dummies(X_cat,drop_first=True)
    X=pd.concat([X_cat_encoded,X_num],axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2,train_size=0.8, random_state = 100)
    
    StandardTransformer = StandardScaler().fit(X_train)
    X_stand = pd.DataFrame(StandardTransformer.transform(X_train), columns = X.columns)
    X_stand_test = pd.DataFrame(StandardTransformer.transform(X_test), columns = X.columns)
    
    clf=RandomForestRegressor(n_estimators=20, criterion='mse', max_depth=15 , min_samples_split=2 )
    clf.fit(X_stand,y_train)
    predictions=clf.predict(X_stand_test)
    r21=clf.score(X_stand,y_train)
    r22=clf.score(X_stand_test,y_test)
    
    def r2(rf, X_train, y_train):
        return r2_score(y_train, rf.predict(X_train))

    perm = permutation_importances(clf, X_train, y_train, r2).reset_index()
    
    feature=[]
    value=[]
    for i in perm['Feature']:
        if ('_') in str(i):
            feature.append(i.split('_')[0])
            value.append(i.split('_')[1])
        else:
            feature.append(i)
            value.append(np.nan)
    
    perm=perm.drop(['Feature'],axis=1)
    perm['feature']=feature
    perm=perm.groupby(['feature']).agg({'Importance':sum}).sort_values(by='Importance',ascending=False).reset_index()
    clean=df.sort_values(by='Rating',ascending=False)
    
    imp_feature=[]
    top_values=[]
    output=[]

    for i in range(5):
        imp_feature.append(perm['feature'][i])
        for n in range(5):
            top_values.append(clean[perm['feature'][i]][n])
    list_of_lists = [top_values[i:i+n] for i in range(0, len(top_values), 5)]

    output.append(dict(zip(imp_feature,list_of_lists)))
    return output

In [6]:
df=pd.read_csv('casco.csv').drop(['Unnamed: 0'],axis=1)

In [193]:
clean=clean(df)

In [5]:
clean.to_csv('clean.csv')

In [6]:
X=clean.drop(['Rating'],axis=1)
y=clean['Rating']

X_cat=X.select_dtypes(include='object')
X_num=X.select_dtypes(include='number')
X_cat_encoded= pd.get_dummies(X_cat,drop_first=True)
X=pd.concat([X_cat_encoded,X_num],axis=1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 100)

In [8]:
StandardTransformer = StandardScaler().fit(X_train)
X_stand = pd.DataFrame(StandardTransformer.transform(X_train), columns = X.columns)
X_stand_test = pd.DataFrame(StandardTransformer.transform(X_test), columns = X.columns)

In [9]:
def train_models(training_x, training_y, testing_x, testing_y, models):

    models_names = list()
    training_scores = list()
    testing_scores = list()
    rmse_scores_training = list()
    rmse_scores_test = list()

    for model in models:

        model.fit(training_x, training_y)
        predicitons_training = model.predict(training_x)
        predictions_test = model.predict(testing_x)
        models_names.append(type(model).__name__)
        training_scores.append(round(model.score(training_x, training_y), 2))
        testing_scores.append(round(model.score(testing_x, testing_y), 2))
        rmse_scores_training.append(mean_squared_error(training_y, predicitons_training, squared = False))
        rmse_scores_test.append(mean_squared_error(testing_y, predictions_test, squared = False))

    results = pd.DataFrame()

    results['model_name'] = models_names
    results['training_score'] = training_scores
    results['testing_score'] = testing_scores
    results['rmse_training'] = rmse_scores_training
    results['rmse_test'] = rmse_scores_test

    return results

models_to_test = [LinearRegression(), XGBRegressor(), RandomForestRegressor(), KNeighborsRegressor(), GradientBoostingRegressor(), CatBoostRegressor(verbose = 0)]

models_results = train_models(X_train, y_train, X_test, y_test, models_to_test)

In [10]:
models_results

,model_name,training_score,testing_score,rmse_training,rmse_test
0,LinearRegression,0.96,-1.956750e+14,0.040494,3.225959e+06
1,XGBRegressor,1.00,8.600000e-01,0.000699,8.741215e-02
2,RandomForestRegressor,0.99,8.600000e-01,0.024373,8.658773e-02
3,KNeighborsRegressor,0.87,5.200000e-01,0.072291,1.598992e-01
4,GradientBoostingRegressor,1.00,8.500000e-01,0.007859,8.852225e-02
5,CatBoostRegressor,1.00,8.700000e-01,0.001826,8.454009e-02


In [11]:
#Choosing Random Forests

In [12]:
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.model_selection import GridSearchCV

n_estimators_choices= [100,50,200] 
max_depth_choices=[0,5,10] 
criterion_choices=['squared_error','mse', 'absolute_error']
min_samples_split_choices=[2,4]

grid={'n_estimators':n_estimators_choices, 'max_depth':max_depth_choices, 'min_samples_split':min_samples_split_choices, 'criterion': criterion_choices} 
model=RandomForestRegressor() 
grid_search=GridSearchCV(estimator=model,param_grid=grid,cv=5) 
grid_search.fit(X_train,y_train) 
grid_search.best_params_

{'criterion': 'mse',
 'max_depth': 10,
 'min_samples_split': 2,
 'n_estimators': 200}

In [13]:
from sklearn.model_selection import cross_validate
results=cross_validate(RandomForestRegressor(n_estimators=20, criterion='mse', max_depth=10 , min_samples_split=2 ),X_train,y_train,cv=5)

In [14]:
results

{'fit_time': array([0.02291393, 0.01690817, 0.01594687, 0.01694322, 0.01594663]),
 'score_time': array([0.0019877 , 0.00298977, 0.00199318, 0.00199342, 0.00199413]),
 'test_score': array([0.92896142, 0.8105372 , 0.92331442, 0.89760081, 0.99497364])}

In [15]:
results['test_score'].mean()

0.9110774999204511

In [16]:
clf=RandomForestRegressor(n_estimators=20, criterion='mse', max_depth=15 , min_samples_split=2 )
clf.fit(X_train,y_train)
predictions=clf.predict(X_test)
r21=clf.score(X_train,y_train)
r22=clf.score(X_test,y_test)
print(mean_squared_error(y_test,predictions,squared = False))

0.09416226557419119


In [17]:
from sklearn.metrics import r2_score
from rfpimp import permutation_importances

def r2(rf, X_train, y_train):
    return r2_score(y_train, rf.predict(X_train))

perm = permutation_importances(clf, X_train, y_train, r2).reset_index()

In [18]:
perm

,Feature,Importance
0,Color_Verde,0.301669
1,Marca_LAKEN,0.196607
2,Price,0.103828
3,Material_Compuesto,0.099471
4,Color_750ml Negro + Funda Térmica,0.044334
...,...,...
56,Color_05. Cobre,0.000000
57,Marca_Super Sparrow,0.000000
58,Marca_Proworks,0.000000
59,Marca_Grsta,0.000000


In [19]:
feature=[]
value=[]
for i in perm['Feature']:
    if ('_') in str(i):
        feature.append(i.split('_')[0])
        value.append(i.split('_')[1])
    else:
        feature.append(i)
        value.append(np.nan)

In [20]:
perm=perm.drop(['Feature'],axis=1)
perm['feature']=feature
perm['value']=value

In [21]:
perm1=perm.drop(['value'],axis=1)

In [22]:
perm2=perm1.groupby(['feature']).agg({'Importance':sum}).sort_values(by='Importance',ascending=False).reset_index()

In [23]:
perm2

,feature,Importance
0,Color,0.395725
1,Marca,0.214219
2,Material,0.150112
3,Price,0.103828
4,Fabricante,0.033904
5,peso gramos,0.016406
6,ancho cm,0.009475
7,alto cm,0.006673
8,Pas de origen,0.005756
9,Departamento,0.002760


In [24]:
clean1=clean.sort_values(by='Rating',ascending=False)

In [25]:
clean1

,Marca,Color,Material,Rating,Price,Fabricante,Departamento,Tamaño,Pas de origen,largo cm,ancho cm,alto cm,peso gramos
92,Milu,Menta,Acero inoxidable,4.8,22.95,Laken,Unisex adulto,‎1000 ml,China,7.00,7.00,23.50,290.00
65,Milu,Menta,Acero inoxidable,4.8,22.95,Laken,Unisex adulto,‎1000 ml,China,7.00,7.00,23.50,290.00
48,Milu,Menta,Acero inoxidable,4.8,22.95,Laken,Unisex adulto,‎1000 ml,China,7.00,7.00,23.50,290.00
49,FLES,Negro Básico,Tritan sin BPA.,4.8,22.95,‎Forsit,Unisex adulto,‎64 oz,China,30.48,12.19,12.19,540.00
34,Milu,Menta,Acero inoxidable,4.8,22.95,Laken,Unisex adulto,‎1000 ml,China,7.00,7.00,23.50,290.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,LAKEN,Verde,Aluminio,4.1,26.09,Laken,Unisex adulto,‎1000 ml,China,7.50,10.00,22.00,240.00
166,LAKEN,Granite Hit,Aluminio,4.0,14.10,Laken,Adultos unisex,‎1000 ml,China,20.00,8.00,24.99,158.76
233,NAWA Home & Work,Amarilla,Acero inoxidable,4.0,14.10,‎Schou Company A/S,Unisex adulto,‎64 oz,España,20.00,6.50,1000.00,170.00
206,LAKEN,Granite Hit,Aluminio,4.0,14.10,Laken,Adultos unisex,‎64 oz,China,9.00,8.00,24.99,158.76


In [27]:
imp_feature=[]
top_values=[]
output=[]

for i in range(6):
    imp_feature.append(perm2['feature'][i])
    for n in range(6):
        top_values.append(clean1[perm2['feature'][i]][n])
list_of_lists = [top_values[i:i+n] for i in range(0, len(top_values), 5)]
print(list_of_lists)
output.append(dict(zip(imp_feature,list_of_lists)))
      
    

[['Negro Básico', '02 Stone Gray | Gris', 'Verde', '13. Lima Lila', 'Rojo 127'], ['Azul Jannu+classic', 'FLES', '720°DGREE', 'Cosumy', 'GadHome'], ['LAKEN', 'LAKEN', 'Tritan sin BPA.', 'Acero inoxidable', 'Acero inoxidable'], ['Acero inoxidable, Cobre', 'Compuesto', 'Acero inoxidable', 14.99, 29.97], [15.73, 19.99, 19.4, 29.5, '\u200eForsit'], ['720°DGREE', '720°DGREE', '720°DGREE', 'Laken', 'Laken'], [540.0, 430.0, 620.0, 620.0, 259.99], [259.99]]


In [28]:
output

[{'Color': ['Negro Básico',
   '02 Stone Gray | Gris',
   'Verde',
   '13. Lima Lila',
   'Rojo 127'],
  'Marca': ['Azul Jannu+classic', 'FLES', '720°DGREE', 'Cosumy', 'GadHome'],
  'Material': ['LAKEN',
   'LAKEN',
   'Tritan sin BPA.',
   'Acero inoxidable',
   'Acero inoxidable'],
  'Price': ['Acero inoxidable, Cobre',
   'Compuesto',
   'Acero inoxidable',
   14.99,
   29.97],
  'Fabricante': [15.73, 19.99, 19.4, 29.5, '\u200eForsit'],
  'peso gramos': ['720°DGREE', '720°DGREE', '720°DGREE', 'Laken', 'Laken']}]

In [29]:
df1=pd.read_csv('casco.csv').drop(['Unnamed: 0'],axis=1)

In [30]:
df1

,Dimensiones del paquete,Producto en Amazon.es desde,Fabricante,ASIN,Nmero de modelo del producto,Pas de origen,Departamento,Clasificacin en los ms vendidos de Amazon,Tamaño,Color,Usos recomendados para el producto,Marca,Tipo de servicio del vehículo,Material del interior,Material para exteriores,Rating,Price,Is Discontinued By Manufacturer,Dimensiones del producto,Peso,Dimensiones del producto: largo x ancho x alto,Rango de edad (descripción),Pilas,Referencia del fabricante,Número de producto,Pilas / baterías incluidas,Pilas / baterías necesarias,Composición de la pila de la batería,Tipo de conector,Peso del producto,Valoración media de los clientes,Clasificación en los más vendidos de Amazon,Nivel de resistencia al agua,Tecnología de conectividad,Modelo,Factor de forma,Función especial,Incluye cubierta,Memoria extraíble,Aparatos compatibles,Tamaño de la pantalla,Tiene estabilizador de imagen,Resolución máxima,Resolución de captura de vídeo,Potencia eléctrica,Restricciones de envío,Material,Tipo de ajuste,Vataje
0,36.9 x 28.2 x 26.1 cm; 2.22 kilogramos,23 diciembre 2020,"Lanxi Yema Motorcycle Fittings Co.,LTD",B07P5Q3GGZ,YEMA-829WXL,China,Niños,Ver el Top 100 en Coche y moto,X-Large,Blanco,Motociclismo,YEMA Helmet,"Scooter, Moto de nieve, Ciclomotor","Poliestireno expandido, ABS",Acrilonitrilo butadieno estireno,4.5,64.99,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36.9 x 28.2 x 26.1 cm; 2.22 kilogramos,19 abril 2017,"Lanxi Yema Motorcycle Fittings Co.,LTD",B071CL27W9,YM-831MBXL,China,Hombre,Ver el Top 100 en Coche y moto,X-Large,Negro Mate,"Snowboarding, Motociclismo, Ciclismo",YEMA Helmet,"Scooter, Moto de nieve, Ciclomotor",Poliestireno expandido,Plástico ABS.,4.5,79.99,No,36 x 27 x 27 cm; 1.6 kilogramos,1.6 Kilogramos,36 x 27 x 27 centímetros,Adulto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36.9 x 28.2 x 26.1 cm; 2.22 kilogramos,19 mayo 2017,"Lanxi Yema Motorcycle Fittings Co.,LTD",B01N3K6V25,YEMA-926MBS,China,Hombres,Ver el Top 100 en Coche y moto,Small,Negro Mate,"Motociclismo, Ciclismo",YEMA Helmet,Moto de nieve,Poliestireno expandido,Acrilonitrilo butadieno estireno,4.4,99.99,No,36.07 x 26.92 x 26.92 cm; 1.7 kilogramos,1.7 Kilogramos,36.1 x 26.9 x 26.9 centímetros,Adulto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,36.3 x 27.5 x 25.5 cm; 2.29 kilogramos,9 diciembre 2020,"Lanxi Yema Motorcycle Fittings Co.,LTD",B08FWXQQJD,YEMA-925WXL-2,China,Niños,Ver el Top 100 en Coche y moto,X-Large,Blanco,Motociclismo,YEMA Helmet,"Scooter, Moto de nieve, Ciclomotor","ABS, Poliestireno expandido",Acrilonitrilo butadieno estireno,3.3,79.99,No,36.07 x 26.92 x 26.92 cm; 1.7 kilogramos,1.7 Kilogramos,36.1 x 26.9 x 26.9 centímetros,Adulto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,‎15.39 x 11.2 x 6.4 cm; 372 gramos,29 julio 2020,‎QSPORTPEAK,B08DTY2T8V,YEMA-925WXL-2,China,Niños,Ver el Top 100 en Coche y moto,X-Large,Blanco,Motociclismo,QSPORTPEAK,"Scooter, Moto de nieve, Ciclomotor","ABS, Poliestireno expandido",Acrilonitrilo butadieno estireno,3.5,71.99,No,36.07 x 26.92 x 26.92 cm; 1.7 kilogramos,1.7 Kilogramos,36.1 x 26.9 x 26.9 centímetros,Adulto,‎1 Litio Ion necesaria,‎M2,‎M2,‎Sí,‎Sí,‎Litio Ion,Inalámbrico,‎372 g,"3,5 de 5 estrellas64 valoraciones3,5 de 5 estr...",nº225 en Manos libres Bluetooth para moto,Impermeable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,36.5 x 27.7 x 26 cm; 2.22 kilogramos,4 abril 2019,INTERPHONE,B07QB3Z3Q9,INTERPHOACTIVETP,Filipinas,Hombres,Manos libres Bluetooth para moto,Small,Negro,"Motociclismo, Ciclismo",I